In [1]:
# Gerekli kütüphaneleri içe aktar
import pandas as pd
import torch
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# CSV dosyasını yükle
df = pd.read_csv('C:/Users/sonay/Desktop/kaggle/e-ticaret_urun_yorumlari.csv', delimiter=';')

# Gerekli sütunları seç ve boş değerleri temizle
df = df[['Metin', 'Durum']]
df.dropna(inplace=True)

# Veriyi eğitim ve doğrulama setlerine ayır
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Metin'].tolist(), df['Durum'].tolist(), test_size=0.2, random_state=42
)

# Tokenizer oluştur
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Dataset sınıfı tanımla
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Eğitim ve doğrulama dataset'lerini oluştur
train_dataset = ReviewDataset(train_texts, train_labels)
val_dataset = ReviewDataset(val_texts, val_labels)

# BERT modelini yükle
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# TrainingArguments oluştur
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # Fine-tuning için epoch sayısı
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch'
)

# Trainer oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Fine-tuning adımı
trainer.train()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\sonay\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.451200,0.352477
2,0.148000,0.303004
3,0.151200,0.315818


TrainOutput(global_step=2277, training_loss=0.3532347969962779, metrics={'train_runtime': 14267.1553, 'train_samples_per_second': 2.552, 'train_steps_per_second': 0.16, 'total_flos': 2394858328160256.0, 'train_loss': 0.3532347969962779, 'epoch': 3.0})

In [2]:
# Modeli kaydet
trainer.save_model("./saved_model")

# Tokenizer'ı kaydet
tokenizer.save_pretrained("./saved_model")


('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\vocab.txt',
 './saved_model\\added_tokens.json')

In [7]:
def predict_sentiment(text):
    # Cümleyi tokenleştir
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    
    # Model ile tahmin yap
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    # En yüksek skora sahip sınıfı seç
    prediction = torch.argmax(logits, dim=1).item()
    
    # Sınıf etiketlerini belirle (0, 1, 2 gibi)
    label_map = {0: "Negatif", 1: "Pozitif", 2: "Nötr"}
    
    return label_map[prediction]


In [8]:
print(predict_sentiment("Bu ürün harika, çok memnun kaldım!"))     # Pozitif beklenir
print(predict_sentiment("Ürün fena değil, idare eder."))            # Nötr beklenir
print(predict_sentiment("Kötü bir ürün, hiç beğenmedim."))          # Negatif beklenir



Pozitif
Nötr
Negatif


In [23]:
print(predict_sentiment("çok şık ve rahat bir ürün "))    

Pozitif
